In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from math import sqrt
import datetime
import plotly.graph_objects as go
from sklearn.cluster import KMeans

/var/folders/px/jj5lh8jj2zv5m3v0dpk80psw0000gn/T/ipykernel_78223/945955947.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


The dataset I have from citibike is way too big, containing almost 40 million observations. It's also split into a collection of different csv files. Here I'll get all the csv files into one, sample the df down to a more manageable size, and save to one dataframe.

I also want to create a separate dataset of daily ride counts before I down sample the data. We can attempt to predict overall daily citibike demand and see how the things that help predict this may differ from those that predict ride duration.

In [15]:
#load in each csv and add it to one really big csv
parse_dates = ['started_at', 'ended_at']
dtypes = {'ride_id': str, 'rideable_type': str, 'start_station_name': str, 'start_station_id': str, 'end_station_name': str, 'end_station_id': str, 'start_lat': float, 'start_lng': float, 'end_lat': float, 'end_lng': float, 'member_casual': str}
#january
df1 = pd.read_csv('data/2023-citibike-tripdata/1_January/202301-citibike-tripdata_1.csv', dtype=dtypes, parse_dates=parse_dates)
df2 = pd.read_csv('data/2023-citibike-tripdata/1_January/202301-citibike-tripdata_2.csv', dtype=dtypes, parse_dates=parse_dates)
#february
df3 = pd.read_csv('data/2023-citibike-tripdata/2_February/202302-citibike-tripdata_1.csv', dtype=dtypes, parse_dates=parse_dates)
df4 = pd.read_csv('data/2023-citibike-tripdata/2_February/202302-citibike-tripdata_2.csv', dtype=dtypes, parse_dates=parse_dates)
#march
df5 = pd.read_csv('data/2023-citibike-tripdata/3_March/202303-citibike-tripdata_1.csv', dtype=dtypes, parse_dates=parse_dates)
df6 = pd.read_csv('data/2023-citibike-tripdata/3_March/202303-citibike-tripdata_2.csv', dtype=dtypes, parse_dates=parse_dates)
df7 = pd.read_csv('data/2023-citibike-tripdata/3_March/202303-citibike-tripdata_3.csv', dtype=dtypes, parse_dates=parse_dates)
#april
df8 = pd.read_csv('data/2023-citibike-tripdata/4_April/202304-citibike-tripdata_1.csv', dtype=dtypes, parse_dates=parse_dates)
df9 = pd.read_csv('data/2023-citibike-tripdata/4_April/202304-citibike-tripdata_2.csv', dtype=dtypes, parse_dates=parse_dates)
df10 = pd.read_csv('data/2023-citibike-tripdata/4_April/202304-citibike-tripdata_3.csv', dtype=dtypes, parse_dates=parse_dates)
#may
df11 = pd.read_csv('data/2023-citibike-tripdata/5_May/202305-citibike-tripdata_1.csv', dtype=dtypes, parse_dates=parse_dates)
df12 = pd.read_csv('data/2023-citibike-tripdata/5_May/202305-citibike-tripdata_2.csv', dtype=dtypes, parse_dates=parse_dates)
df13 = pd.read_csv('data/2023-citibike-tripdata/5_May/202305-citibike-tripdata_3.csv', dtype=dtypes, parse_dates=parse_dates)
df14 = pd.read_csv('data/2023-citibike-tripdata/5_May/202305-citibike-tripdata_4.csv', dtype=dtypes, parse_dates=parse_dates)
#june
df15 = pd.read_csv('data/2023-citibike-tripdata/6_June/202306-citibike-tripdata_1.csv', dtype=dtypes, parse_dates=parse_dates)
df16 = pd.read_csv('data/2023-citibike-tripdata/6_June/202306-citibike-tripdata_2.csv', dtype=dtypes, parse_dates=parse_dates)
df17 = pd.read_csv('data/2023-citibike-tripdata/6_June/202306-citibike-tripdata_3.csv', dtype=dtypes, parse_dates=parse_dates)
df18 = pd.read_csv('data/2023-citibike-tripdata/6_June/202306-citibike-tripdata_4.csv', dtype=dtypes, parse_dates=parse_dates)
#july
df19 = pd.read_csv('data/2023-citibike-tripdata/7_July/202307-citibike-tripdata_1.csv', dtype=dtypes, parse_dates=parse_dates)
df20 = pd.read_csv('data/2023-citibike-tripdata/7_July/202307-citibike-tripdata_2.csv', dtype=dtypes, parse_dates=parse_dates)
df21 = pd.read_csv('data/2023-citibike-tripdata/7_July/202307-citibike-tripdata_3.csv', dtype=dtypes, parse_dates=parse_dates)
df22 = pd.read_csv('data/2023-citibike-tripdata/7_July/202307-citibike-tripdata_4.csv', dtype=dtypes, parse_dates=parse_dates)
#august
df23 = pd.read_csv('data/2023-citibike-tripdata/8_August/202308-citibike-tripdata_1.csv', dtype=dtypes, parse_dates=parse_dates)
df24 = pd.read_csv('data/2023-citibike-tripdata/8_August/202308-citibike-tripdata_2.csv', dtype=dtypes, parse_dates=parse_dates)
df25 = pd.read_csv('data/2023-citibike-tripdata/8_August/202308-citibike-tripdata_3.csv', dtype=dtypes, parse_dates=parse_dates)
df26 = pd.read_csv('data/2023-citibike-tripdata/8_August/202308-citibike-tripdata_4.csv', dtype=dtypes, parse_dates=parse_dates)
df27 = pd.read_csv('data/2023-citibike-tripdata/8_August/202308-citibike-tripdata_5.csv', dtype=dtypes, parse_dates=parse_dates)
#september
df28 = pd.read_csv('data/2023-citibike-tripdata/9_September/202309-citibike-tripdata_1.csv', dtype=dtypes, parse_dates=parse_dates)
df29 = pd.read_csv('data/2023-citibike-tripdata/9_September/202309-citibike-tripdata_2.csv', dtype=dtypes, parse_dates=parse_dates)
df30 = pd.read_csv('data/2023-citibike-tripdata/9_September/202309-citibike-tripdata_3.csv', dtype=dtypes, parse_dates=parse_dates)
df31 = pd.read_csv('data/2023-citibike-tripdata/9_September/202309-citibike-tripdata_4.csv', dtype=dtypes, parse_dates=parse_dates)
#october
df32 = pd.read_csv('data/2023-citibike-tripdata/10_October/202310-citibike-tripdata_1.csv', dtype=dtypes, parse_dates=parse_dates)
df33 = pd.read_csv('data/2023-citibike-tripdata/10_October/202310-citibike-tripdata_2.csv', dtype=dtypes, parse_dates=parse_dates)
df34 = pd.read_csv('data/2023-citibike-tripdata/10_October/202310-citibike-tripdata_3.csv', dtype=dtypes, parse_dates=parse_dates)
df35 = pd.read_csv('data/2023-citibike-tripdata/10_October/202310-citibike-tripdata_4.csv', dtype=dtypes, parse_dates=parse_dates)
#november
df36 = pd.read_csv('data/2023-citibike-tripdata/11_November/202311-citibike-tripdata_1.csv', dtype=dtypes, parse_dates=parse_dates)
df37 = pd.read_csv('data/2023-citibike-tripdata/11_November/202311-citibike-tripdata_2.csv', dtype=dtypes, parse_dates=parse_dates)
df38 = pd.read_csv('data/2023-citibike-tripdata/11_November/202311-citibike-tripdata_3.csv', dtype=dtypes, parse_dates=parse_dates)
#december
df39 = pd.read_csv('data/2023-citibike-tripdata/12_December/202312-citibike-tripdata_1.csv', dtype=dtypes, parse_dates=parse_dates)
df40 = pd.read_csv('data/2023-citibike-tripdata/12_December/202312-citibike-tripdata_2.csv', dtype=dtypes, parse_dates=parse_dates)
df41 = pd.read_csv('data/2023-citibike-tripdata/12_December/202312-citibike-tripdata_3.csv', dtype=dtypes, parse_dates=parse_dates)


#concatenate all the dataframes
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20, df21, df22, df23, df24, df25, df26, df27, df28, df29, df30, df31, df32, df33, df34, df35, df36, df37, df38, df39, df40, df41])
#remove rows with missing values
df = df.dropna()
# create a new dataframe with the daily counts of total rides
total_rides = df['started_at'].dt.date.value_counts().sort_index()
total_rides = total_rides.reset_index()
total_rides.columns = ['date', 'total_rides']
# save the daily counts to a csv
total_rides.to_csv('data/2023-citibike-tripdata/2023-citibike-tripdata-daily-counts.csv')

# randomly sample 20 percent of the data
df = df.sample(frac=0.05)


#save the big csv
df.to_csv('data/2023-citibike-tripdata/2023-citibike-tripdata.csv')

KeyboardInterrupt: 

Here I'll begin adding some additional data to the dataset that may help predict ride duration. This includes some weather data and data on other forms of transit in NYC.

In [17]:
parse_dates = ['started_at', 'ended_at']
dtypes = {'ride_id': str, 'rideable_type': str, 'start_station_name': str, 'start_station_id': str, 'end_station_name': str, 'end_station_id': str, 'start_lat': float, 'start_lng': float, 'end_lat': float, 'end_lng': float, 'member_casual': str}

# read in the citibike dataset
df = pd.read_csv('data/2023-citibike-tripdata/2023-citibike-tripdata.csv', dtype=dtypes, parse_dates=parse_dates)
# read in the daily counts of total rides
total_rides = pd.read_csv('data/2023-citibike-tripdata/2023-citibike-tripdata-daily-counts.csv', parse_dates=['date'])

# Weather
# read in the weather data
weather = pd.read_csv('data/weather/new york city 2023-01-01 to 2023-12-31.csv', parse_dates=['datetime'])
# keep the datetime, temp, feelslike, dew, humidity, conditions, visibility, cloudcover, windspeed, precip columns 
weather = weather[['datetime', 'temp', 'feelslike', 'dew', 'humidity', 'conditions', 'visibility', 'cloudcover', 'windspeed', 'precip']]
# merge the two dataframes
df = pd.merge_asof(df.sort_values('started_at'), weather.sort_values('datetime'), left_on='started_at', right_on='datetime', direction='nearest')
# merge the weather and total_rides dataframes
total_rides = pd.merge_asof(total_rides.sort_values('date'), weather.sort_values('datetime'), left_on='date', right_on='datetime', direction='nearest')

# MTA
# read in the mta data
mta = pd.read_csv('data/transit/MTA_Daily_Ridership_Data__Beginning_2020_20240305.csv', parse_dates=['Date'])
# keep only the columns we need in MTA
mta = mta[['Date', 'Staten Island Railway: Total Estimated Ridership', 'Bridges and Tunnels: Total Traffic', 'Access-A-Ride: Total Scheduled Trips', 'Metro-North: Total Estimated Ridership', 'LIRR: Total Estimated Ridership', 'Buses: Total Estimated Ridership', 'Subways: Total Estimated Ridership']]
# merge the two dataframes
df = pd.merge_asof(df.sort_values('started_at'), mta.sort_values('Date'), left_on='started_at', right_on='Date', direction='nearest')
# merge the mta and total_rides dataframes
total_rides = pd.merge_asof(total_rides.sort_values('date'), mta.sort_values('Date'), left_on='date', right_on='Date', direction='nearest')
# shorten the column names
df = df.rename(columns={'Staten Island Railway: Total Estimated Ridership': 'SIR', 'Bridges and Tunnels: Total Traffic': 'Bridges_Tunnels', 'Access-A-Ride: Total Scheduled Trips': 'Access_A_Ride', 'Metro-North: Total Estimated Ridership': 'Metro_North', 'LIRR: Total Estimated Ridership': 'LIRR', 'Buses: Total Estimated Ridership': 'Buses', 'Subways: Total Estimated Ridership': 'Subways'})
total_rides = total_rides.rename(columns={'Staten Island Railway: Total Estimated Ridership': 'SIR', 'Bridges and Tunnels: Total Traffic': 'Bridges_Tunnels', 'Access-A-Ride: Total Scheduled Trips': 'Access_A_Ride', 'Metro-North: Total Estimated Ridership': 'Metro_North', 'LIRR: Total Estimated Ridership': 'LIRR', 'Buses: Total Estimated Ridership': 'Buses', 'Subways: Total Estimated Ridership': 'Subways'})

# remove the Date column
df = df.drop(columns=['Date'])
total_rides = total_rides.drop(columns=['Date'])
# remove the date column from total_rides
total_rides = total_rides.drop(columns=['date'])
# remove the datetime column from df
df = df.drop(columns=['datetime'])

# drop the Unnamed: 0 column
df = df.drop(columns=['Unnamed: 0'])
total_rides = total_rides.drop(columns=['Unnamed: 0'])
# drop any rows with missing values
df = df.dropna()
# add ride duration column
df['ride_duration'] = (df['ended_at'] - df['started_at']).dt.total_seconds()
# get rid on any rows where the ride duration is less than 60 seconds because those are likely not true rides
df = df[df['ride_duration'] > 60]

# also want to get a set of unique stations
# run through the start stations and get a unique set of station names, getting lat long for each
stations = df[['start_station_name', 'start_station_id', 'start_lat', 'start_lng']]
# drop duplicates based on station name
stations = stations.drop_duplicates(subset='start_station_id')
# rename start_station_name to station_name and start_station_id to station_id
stations = stations.rename(columns={'start_station_name': 'station_name', 'start_station_id': 'station_id'})
# now lets try doing some clustering of stations
#arbitrarily choose 6 clusters for now
n_clusters = 6
# create a kmeans model
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
# fit the model to the data
kmeans.fit(stations[['start_lat', 'start_lng']])
# add the cluster labels to the stations dataframe
stations['cluster'] = kmeans.labels_
# add the cluster labels to the main dataframe, matching station_id to start_station_id
df = pd.merge(df, stations[['station_id', 'cluster']], left_on='start_station_id', right_on='station_id', how='left')
# drop the station_id column
df = df.drop(columns=['station_id'])


#save the big csv
df.to_csv('data/2023-citibike-full-data.csv')
# save the total_rides dataframe
total_rides.to_csv('data/2023-citibike-tripdata-daily-counts.csv')
# save the stations dataframe
stations.to_csv('data/2023-citibike-stations.csv')



In [5]:
stations = pd.read_csv('data/2023-citibike-stations.csv')

mapbox_access = "pk.eyJ1IjoiZnNoYXBlciIsImEiOiJjbGE4aDZmNjYwMDc1M25wbGtobXM2Y2lrIn0.sZC4RPkqis6KNwGOSSnwkQ"

# Create a map
fig = go.Figure(go.Scattermapbox(
    lat=stations['start_lat'],
    lon=stations['start_lng'],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=5,
        color=stations['cluster'],
        colorscale='Viridis',
        colorbar=dict(title='Cluster')
    ),
))

# Update the layout of the map
fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=40.751999087,
            lon=-73.950742803
        ),
        pitch=0,
        zoom=10.5
    ),
    title = 'Citi Bike Stations in New York City',
    width=800,  # Width of the map
    height=1000   # Height of the map, adjust these dimensions as needed
)

# Display the map
fig.show()

# save the stations dataframe
stations.to_csv('data/2023-citibike-stations.csv')

In [19]:
parse_dates = ['started_at', 'ended_at']
dtypes = {'ride_id': str, 'rideable_type': str, 'start_station_name': str, 'start_station_id': str, 'end_station_name': str, 'end_station_id': str, 'start_lat': float, 'start_lng': float, 'end_lat': float, 'end_lng': float, 'member_casual': str}

# read in the citibike dataset
df = pd.read_csv('data/2023-citibike-full-data.csv', dtype=dtypes, parse_dates=parse_dates)
# read in the daily counts of total rides
total_rides = pd.read_csv('data/2023-citibike-tripdata-daily-counts.csv', parse_dates=['datetime'])

# now gonna go through and add some dummies to the data
# add a dummy for if the ride was on a weekday
df['weekday'] = df['started_at'].dt.dayofweek
df['weekday'] = np.where(df['weekday'] < 5, 1, 0)
total_rides['weekday'] = total_rides['datetime'].dt.dayofweek
total_rides['weekday'] = np.where(total_rides['weekday'] < 5, 1, 0)
# a dummy for if the ride was on a holiday
# holidays created from list of federal holidays in 2023
holidays = ['2023-01-02', '2023-01-16', '2023-02-20', '2023-05-29', '2023-07-04', '2023-09-04', '2023-10-09', '2023-11-10', '2023-11-23', '2023-12-25']
df['holiday'] = np.where(df['started_at'].dt.date.astype(str).isin(holidays), 1, 0)
total_rides['holiday'] = np.where(total_rides['datetime'].astype(str).isin(holidays), 1, 0)
# add data for what month the ride was in
df['month'] = df['started_at'].dt.month
# convert it to a float
df['month'] = df['month'].astype(float)
total_rides['month'] = total_rides['datetime'].dt.month
# convert it to a float
total_rides['month'] = total_rides['month'].astype(float)

# add a dummy for precipitation
df['precip_d'] = np.where(df['precip'] > 0, 1, 0)
total_rides['precip_d'] = np.where(total_rides['precip'] > 0, 1, 0)
#dummy for if temperature is below 40
df['cold_d'] = np.where(df['temp'] < 40, 1, 0)
total_rides['cold_d'] = np.where(total_rides['temp'] < 40, 1, 0)
#dummy for if temperature is above 90
df['hot_d'] = np.where(df['temp'] > 90, 1, 0)
total_rides['hot_d'] = np.where(total_rides['temp'] > 90, 1, 0)

# add dummy for night rides later than 10 pm and before 6 am
df['night'] = np.where((df['started_at'].dt.hour > 22) | (df['started_at'].dt.hour < 6), 1, 0)

# add a dummy for each cluster of stations
df['station0'] = np.where(df['cluster'] == 0, 1, 0)
df['station1'] = np.where(df['cluster'] == 1, 1, 0)
df['station2'] = np.where(df['cluster'] == 2, 1, 0)
df['station3'] = np.where(df['cluster'] == 3, 1, 0)
df['station4'] = np.where(df['cluster'] == 4, 1, 0)
df['station5'] = np.where(df['cluster'] == 5, 1, 0)

# add a dummy for if the ride was a member or casual
df['member'] = np.where(df['member_casual'] == 'member', 1, 0)
# add a dummy for electric bikes
df['electric'] = np.where(df['rideable_type'] == 'electric_bike', 1, 0)

# add a dummy for each of the other transportation elements I have 
# make the dummy = 1 if the value is greater than x standard deviations from the mean
# bridges and tunnels
df['Bridges_Tunnels_d'] = np.where(df['Bridges_Tunnels'] > df['Bridges_Tunnels'].mean() + df['Bridges_Tunnels'].std(), 1, 0)
total_rides['Bridges_Tunnels_d'] = np.where(total_rides['Bridges_Tunnels'] > total_rides['Bridges_Tunnels'].mean() + total_rides['Bridges_Tunnels'].std(), 1, 0)
# access a ride
df['Access_A_Ride_d'] = np.where(df['Access_A_Ride'] > df['Access_A_Ride'].mean() + df['Access_A_Ride'].std(), 1, 0)
total_rides['Access_A_Ride_d'] = np.where(total_rides['Access_A_Ride'] > total_rides['Access_A_Ride'].mean() + total_rides['Access_A_Ride'].std(), 1, 0)
# metro north
df['Metro_North_d'] = np.where(df['Metro_North'] > df['Metro_North'].mean() + df['Metro_North'].std(), 1, 0)
total_rides['Metro_North_d'] = np.where(total_rides['Metro_North'] > total_rides['Metro_North'].mean() + total_rides['Metro_North'].std(), 1, 0)
# LIRR
df['LIRR_d'] = np.where(df['LIRR'] > df['LIRR'].mean() + df['LIRR'].std(), 1, 0)
total_rides['LIRR_d'] = np.where(total_rides['LIRR'] > total_rides['LIRR'].mean() + total_rides['LIRR'].std(), 1, 0)
# buses
df['Buses_d'] = np.where(df['Buses'] > df['Buses'].mean() + df['Buses'].std(), 1, 0)
total_rides['Buses_d'] = np.where(total_rides['Buses'] > total_rides['Buses'].mean() + total_rides['Buses'].std(), 1, 0)
# subways
df['Subways_d'] = np.where(df['Subways'] > df['Subways'].mean() + df['Subways'].std(), 1, 0)
total_rides['Subways_d'] = np.where(total_rides['Subways'] > total_rides['Subways'].mean() + total_rides['Subways'].std(), 1, 0)

# remove the Unnamed: 0 column
df = df.drop(columns=['Unnamed: 0'])
total_rides = total_rides.drop(columns=['Unnamed: 0'])

# save the big csv
df.to_csv('data/2023-citibike-full-data.csv')
# save the total_rides dataframe
total_rides.to_csv('data/2023-citibike-tripdata-daily-counts.csv')

# pint columns
print(df.columns)

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'temp', 'feelslike', 'dew', 'humidity', 'conditions',
       'visibility', 'cloudcover', 'windspeed', 'precip', 'SIR',
       'Bridges_Tunnels', 'Access_A_Ride', 'Metro_North', 'LIRR', 'Buses',
       'Subways', 'ride_duration', 'cluster', 'weekday', 'holiday', 'month',
       'precip_d', 'cold_d', 'hot_d', 'night', 'station0', 'station1',
       'station2', 'station3', 'station4', 'station5', 'member', 'electric',
       'Bridges_Tunnels_d', 'Access_A_Ride_d', 'Metro_North_d', 'LIRR_d',
       'Buses_d', 'Subways_d'],
      dtype='object')
